In [ ]:
from hig_data.coco import COCOStuffDataset, CocoStuffGraphDataset
import torch
img_path = '/Users/rupertmenneer/Downloads/coco_val2017_256-sd.zip'
mask_path = '/Users/rupertmenneer/Downloads/coco_val2017_masks_256.zip'
coco_graph = CocoStuffGraphDataset(img_path, mask_path, latent_images=True)
data = coco_graph[4]
data

In [3]:
from training.networks_edm2_hignn import Precond
precond = Precond(32, 3, gnn_metadata = data.metadata())

In [ ]:
import torch
images = torch.randn(1, 3, 32, 32)
rnd_normal = torch.randn([images.shape[0], 1, 1, 1], device=images.device)
sigma = (rnd_normal).exp()

data = coco_graph[1]
out = precond(images, sigma, data)
out.shape

In [ ]:
!pip install click

In [ ]:
from training.training_loop_hignn import training_loop
import torch
training_loop(device=torch.device('cpu'), dataset_kwargs=dict(class_name='hig_data.coco.CocoStuffGraphDataset',
image_path=img_path, mask_path=mask_path, latent_images=True), cudnn_benchmark=False,)